In [1]:
# We use this function to call bx from Python, see below for an example
import subprocess
def call(cmd):
    res = subprocess.check_output([cmd], shell=True)
    return res.strip().decode()

import base58
import hmac
import hashlib

entropy = "openssl rand -hex 24"

## Warmup: Show that the child private key is the parent private key + L256

In [2]:
# use base58.b58decode to decode the xpub or xprv and get the chaincode bytes

parent_xpriv = call(f'{entropy} | bx mnemonic-new | bx mnemonic-to-seed | bx hd-new')
parent_xpub = call(f'bx hd-to-public {parent_xpriv}')
parent_priv = call(f'bx hd-to-ec {parent_xpriv}')
child_xpriv = call(f'bx hd-private --index 0 {parent_xpriv}')
child_xpub = call(f'bx hd-to-public {child_xpriv}')
child_priv = call(f'bx hd-to-ec {child_xpriv}')

xpub_bytes = base58.b58decode(parent_xpub)
chaincode = xpub_bytes[13:13+32]
parent_pub = call(f'bx hd-to-ec {parent_xpub}')
concatenated = parent_pub + "00000000"

l256 = hmac.new(
    chaincode,
    msg=bytes.fromhex(concatenated),
    digestmod=hashlib.sha512
).hexdigest()[:64]

print(call(f'bx ec-add-secrets {parent_priv} {l256}'))
print(child_priv)

6d70165aabcc79b05b2d407a37ac4ea41fc3832f696d0dfa3cfe7a2c8501c762
6d70165aabcc79b05b2d407a37ac4ea41fc3832f696d0dfa3cfe7a2c8501c762


# Main Exercise: HD Parent Key Exposure

<br>
<img src="images/hd_parent_exposure.jpg" alt="drawing" style="" width="700px"/>



The above figure briefly recaps how parent private keys can be leaked based on child private keys when deriving non-hardened children.

The goal of this exercise is to demonstrate how this works.

If you need some hints, you can check out this implementation in the `pywallet` library:
https://github.com/ranaroussi/pywallet/blob/468622dcf993a27a5b585289b2724986c02a1fbc/pywallet/utils/bip32.py#L380-L429

### Derive new parent key pair

In [3]:
parent_xpriv = call(f'{entropy} | bx mnemonic-new | bx mnemonic-to-seed | bx hd-new')
parent_xpub = call(f'bx hd-to-public {parent_xpriv}')
print(parent_xpriv)
print(parent_xpub)

parent_priv = call(f'bx hd-to-ec {parent_xpriv}')
print(parent_priv)

xprv9s21ZrQH143K2uMTH5ZFgxWBy8ciUCf4ppjbPFV4oRgdhVBXBqsjEi6gcSdaE2sQkUyjzzJx7xLutTGjBuwf3WijoNYvYqqa9TvSyXxchzn
xpub661MyMwAqRbcFPRvP76G46SvXATCsfNvC3fCBdtgMmDcaHWfjPBynWRAThkfeqZedeWH7jU3YL7YCsgfhTak5yvUHvQK5M9jD65Ca7M2DtY
2ccdf0375712a506f2510fde360a9d6f5ec685eb2e860cf41bea205ed3f6ddca


### Derive non-hardenend children

In [4]:
child_xpriv = call(f'bx hd-private --index 0 {parent_xpriv}')
child_priv = call(f'bx hd-to-ec {child_xpriv}')
child_xpub = call(f'bx hd-to-public {child_xpriv}')

print("Exposed material:")
print(f"Parent xpub: {parent_xpub}")
print(f"Child priv: {child_priv}")
print("Child index: 0")

Exposed material:
Parent xpub: xpub661MyMwAqRbcFPRvP76G46SvXATCsfNvC3fCBdtgMmDcaHWfjPBynWRAThkfeqZedeWH7jU3YL7YCsgfhTak5yvUHvQK5M9jD65Ca7M2DtY
Child priv: e8371286dde71e1bf77aab07c61c006a6365ef191c5af9692d6d8cd5a30636a4
Child index: 0


### Step 1: Extract chaincode from parent xpub

In [5]:
xpub_bytes = base58.b58decode(parent_xpub)
chaincode = xpub_bytes[13:13+32]
print(list(chaincode))

[84, 217, 114, 47, 216, 132, 111, 126, 151, 142, 181, 10, 138, 151, 159, 246, 103, 136, 17, 127, 217, 127, 37, 83, 78, 179, 182, 200, 65, 50, 128, 144]


### Step2: Compute L256 bits as HMAC-SHA512(Parent Chain Code, Parent Public Key || Child Index)

In [6]:
parent_pub = call(f'bx hd-to-ec {parent_xpub}')
print(parent_pub)
concatenated = parent_pub + "00000000"

0238e69c26c5cb830a6a05369784f7b71e7327ada6983555e89a58e888a7e9cb10


In [7]:
l256 = hmac.new(
    chaincode,
    msg=bytes.fromhex(concatenated),
    digestmod=hashlib.sha512
).hexdigest()[:64]

print(l256)

bb69224f86d4791505299b29901162fb049f692dedd4ec7511836c76cf0f58da


### Step 3: Compute Parent private key as child private key - L256

In [9]:
from itertools import takewhile

group_order = "fffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f"

group_order_m_l256 = hex(int(group_order, 16) - int(l256, 16))[2:]
print(group_order_m_l256)

parent_priv_computed = call(f'bx ec-add-secrets {child_priv} {group_order_m_l256}')
print(parent_priv_computed)
print(parent_priv)
print(f"The two only share the upper {len(list(takewhile(lambda t: t[0]==t[1], zip(parent_priv, parent_priv_computed))))/2} bytes", u"\U0001F914")

4496ddb0792b86eafad664d66fee9d04fb6096d2122b138aee7c938830f0a355
2ccdf0375712a506f2510fde360a9d70a417a9047f3d6cb85c17c1d103c098b8
2ccdf0375712a506f2510fde360a9d6f5ec685eb2e860cf41bea205ed3f6ddca
The two only share the upper 15.0 bytes 🤔
